##### Copyright 2023 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Promoción de tipos TF-NumPy

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/tf_numpy_type_promotion"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/tf_numpy_type_promotion.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar bloc de notas</a>
</td>
</table>

## Descripción general

Hay 4 opciones de promoción de tipos en TensorFlow.

- Por defecto, TensorFlow arroja errores en vez de promover tipos para operaciones de tipo mezclado.
- Al ejecutar `tf.numpy.experimental_enable_numpy_behavior()`, TensorFlow cambia y usa [reglas de promoción de tipos NumPy](https://www.tensorflow.org/guide/tf_numpy#type_promotion).
- En **este documento** se describen dos opciones nuevas que estarán disponibles en TensorFlow 2.15 (o que ya se encuentran en `tf-nightly`):

In [ ]:
!pip install -q tf_nightly

**Nota**: `experimental_enable_numpy_behavior` cambia el comportamiento de todo lo de TensorFlow.

## Preparación

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.experimental.numpy as tnp

print("Using TensorFlow version %s" % tf.__version__)

### Habilitación de la nueva promoción de tipos

A fin de usar la [promoción de tipos como JAX](https://jax.readthedocs.io/en/latest/type_promotion.html) en TF-Numpy, especifique `'all'` o `'safe'` como modos de conversión de tipo d al habilitar el comportamiento NumPy para TensorFlow.

Este sistema nuevo (con `dtype_conversion_mode="all"`) es asociativo, conmutativo y facilita el control del ancho del flotante que se termina aplicando (no convierte automáticamente a flotantes más anchos). Lo que sí hace es presentar algunos riesgos de desbordamiento o de pérdida de precisión, pero `dtype_conversion_mode="safe"` lo forzará a ocuparse de esos casos explícitamente. Ambos modos se explican con más detalle en la [próxima sección](#two_modes).

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")

<a name="two_modes">
</a>

## Dos modos: modo ALL vs. modo SAFE

Con el sistema de promoción de tipos nueva, presentamos dos modos: el modo `ALL` y el modo `SAFE`. El modo `SAFE` se usa para mitigar las posibles promociones "riesgosas" que podrían resultar a partir de la pérdida de precisión o de la ampliación en bits.

### Tipos d

Para no extendernos, usaremos las siguientes abreviaturas.

- `b` significa `tf.bool`
- `u8` significa `tf.uint8`
- `i16` significa `tf.int16`
- `i32` significa `tf.int32`
- `bf16` significa `tf.bfloat16`
- `f32` significa `tf.float32`
- `f64` significa `tf.float64`
- `i32*` significa `int` en Python o `i32` débilmente tipado.
- `f32*` significa `float` en Python o `f32` débilmente tipado.
- `c128*` significa `complex` en Python o `c128` débilmente tipado.

El asterisco (*) denota que el tipo correspondiente es "débil", como un tipo d que el sistema infiere temporalmente y podría diferir a otros tipos d. Este concepto se explica más en detalle [aquí](#weak_tensor).

### Ejemplo de operaciones de pérdida de precisión

En el siguiente ejemplo, `i32` + `f32` se admite en el modo `ALL` pero no en el modo `SAFE` debido al riesgo de pérdida de precisión.

In [ ]:
# i32 + f32 returns a f32 result in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, dtype = tf.int32)
b = tf.constant(5.0, dtype = tf.float32)
a + b  # <tf.Tensor: shape=(), dtype=float32, numpy=15.0>

In [ ]:
# This promotion is not allowed in SAFE mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="safe")
a = tf.constant(10, dtype = tf.int32)
b = tf.constant(5.0, dtype = tf.float32)
try:
  a + b
except TypeError as e:
   print(f'{type(e)}: {e}')  # TypeError: explicitly specify the dtype or switch to ALL mode.

### Ejemplo de operaciones de ampliación de bits

En el siguiente ejemplo, `i8` + `u32` se admite en el modo `ALL`, pero no en el modo `SAFE` debido a la ampliación de bits, que implica usar más bits de los que hay en las entradas. Tenga en cuenta que la semántica de la promoción de tipos nueva solamente permite la ampliación de bits necesaria.

In [ ]:
# i8 + u32 returns an i64 result in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, dtype = tf.int8)
b = tf.constant(5, dtype = tf.uint32)
a + b

In [ ]:
# This promotion is not allowed in SAFE mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="safe")
a = tf.constant(10, dtype = tf.int8)
b = tf.constant(5, dtype = tf.uint32)
try:
  a + b
except TypeError as e:
   print(f'{type(e)}: {e}')  # TypeError: explicitly specify the dtype or switch to ALL mode.

## Un sistema basado en retículo

### Promoción de tipos basada en retículo

El comportamiento de la promoción de tipos nueva se determina a través de la siguiente promoción de tipos basada en retículo (<em>lattice</em>):

![Promoción de tipo basada en retículo](https://tensorflow.org/guide/images/new_type_promotion/type_promotion_lattice.png)

Para ser más específicos, la promoción entre dos tipos cualquiera está determinada por encontrar el primer hijo común de los dos nodos (incluidos los mismos nodos).

Por ejemplo, en el diagrama anterior, el primer hijo común de `i8` e `i32` es `i32` porque la intersección de los dos nodos se produce por primera vez en `i32`, cuando se sigue la dirección de las flechas.

De un modo similar, como en otro ejemplo, el resultado de la promoción de tipos entre `u64` y `f16` sería `f16`.

<a name="promotion_table">
</a>

### Tabla de promoción de tipos

Después de la promoción basada en retículo se genera la tabla de promoción binaria que se encuentra a continuación:

**Nota**: el modo `SAFE` rechaza las celdas destacadas. El modo `ALL` admite todos los casos.

![Tabla de promoción de tipo](https://tensorflow.org/guide/images/new_type_promotion/type_promotion_table.png)

## Ventajas de la promoción de tipos nueva

Adoptamos un sistema basado en retículo como JAX para nuestra promoción de tipos nueva, que ofrece las siguientes ventajas:

<a name="lattice_system_design">
</a>

#### Ventajas del sistema basado en retículo

Primero, al usar un sistema basado en retículo se garantizan tres propiedades importantes:

- Existencia: Hay un único resultado de la promoción de tipos independientemente de las combinaciones de tipo que se hagan.
- Conmutabilidad: `a + b = b + a`
- Asociatividad: `a + (b + c) = (a + b) = c`

Estas tres propiedades son críticas para construir una semántica para la promoción de tipos que sea consistente y predecible.

#### Ventajas del sistema basado en retículo en JAX o bibliotecas similares

Otra ventaja crucial del sistema basado en retículo en JAX o bibliotecas similares es que fuera de los enteros sin asignar, evita todas las promociones que sean más amplias de lo necesario. Significa que no es posible obtener resultados de 64 bits sin entradas de 64 bits. Esto es particularmente beneficioso para trabajar con aceleradores, ya que evita valores de 64 bits innecesarios, algo que se producía frecuentemente con el tipo anterior de promoción.

Sin embargo, viene con una contrapartida: la promoción de enteros/flotantes combinada es muy propensa a la pérdida de precisión. Observemos, en el ejemplo a continuación, el resultado de `i64` + `f16` es la promoción de `i64` a `f16`.

In [ ]:
# The first input is promoted to f16 in ALL mode.
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
tf.constant(1, tf.int64) + tf.constant(3.2, tf.float16)  # <tf.Tensor: shape=(), dtype=float16, numpy=4.2>

Para mitigar este problema, presentamos un modo `SAFE` que rechazará estas promociones "riesgosas".

**Nota**: Para conocer más acerca de las consideraciones relacionadas con el diseño en la construcción del sistema basado en retículo, consulte la información sobre el [diseño de la semántica de promoción de tipos para JAX](https://jax.readthedocs.io/en/latest/jep/9407-type-promotion.html).

<a name="weak_tensor">
</a>

## WeakTensor

### Descripción general

Los *tensores débiles* son tensores que están "débilmente tipados", similares a un [concepto en JAX](https://jax.readthedocs.io/en/latest/type_promotion.html#weakly-typed-values-in-jax).

El sistema infiere temporalmente el tipo d de `WeakTensor` y podría delegar a otros tipos d. Este concepto se introduce en la nueva promoción de tipos para evitar la promoción de tipos no deseados dentro de operaciones binarias entre valores TF y valores con tipos con usuarios especificados no explícitos, como los escalares literales en Python.

Observemos, en el ejemplo a continuación, `tf.constant(1.2)` es considerada "débil" porque no tiene un tipo d específico. Por lo tanto, `tf.constant(1.2)` delega al tipo `tf.constant(3.1, tf.float16)`, dando como resultado una salida `f16`.

In [ ]:
tf.constant(1.2) + tf.constant(3.1, tf.float16)  # <tf.Tensor: shape=(), dtype=float16, numpy=4.3>

### Construcción de tensores débiles

Los tensores débiles (WeakTensor) se generan a partir de la creación de un tensor sin especificar el tipo d. En esos casos, el resultado es un WeakTensor. Se puede comprobar si un tensor es "débil" (<em>weak</em>) o no, controlando el atributo al final de la representación de la <em>string</em> del tensor.

**Primer caso**: cuando `tf.constant` se invoca con una entrada de tipo d sin usuario especificado.

In [ ]:
tf.constant(5)  # <tf.Tensor: shape=(), dtype=int32, numpy=5, weak=True>

In [ ]:
tf.constant([5.0, 10.0, 3])  # <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 5., 10.,  3.], dtype=float32), weak=True>

In [ ]:
# A normal Tensor is created when dtype arg is specified.
tf.constant(5, tf.int32)  # <tf.Tensor: shape=(), dtype=int32, numpy=5>

**Segundo caso**: cuando una entrada tipo d sin usuario especificado se pasa a una [API compatible con WeakTensor](#weak_tensor_apis).

In [ ]:
tf.math.abs([100.0, 4.0])  # <tf.Tensor: shape=(2,), dtype=float32, numpy=array([100., 4.], dtype=float32), weak=True>

##Efectos de activar la nueva promoción de tipos

A continuación, hay una lista no exhaustiva de los cambios que resultan de la activación de la promoción de tipos nueva.

- Resultados de promoción más consistentes y predecibles.
- Riesgo reducido de ampliación de bits.
- Los métodos mágicos (<em>dunder</em>) de `tf.Tensor` usan la promoción de tipos nueva.
- `tf.constant` puede devolver `WeakTensor`.
- `tf.constant` permite conversiones implícitas cuando una entrada de tensor se pasa con un tipo d diferente del argumento `dtype`.
- Las operaciones (ops) `tf.Variable` en el lugar (`assign`, `assign-add`, `assign-sub`) permiten conversiones implícitas.
- `tnp.array(1)` y `tnp.array(1.0)` devuelve WeakTensor de 32 bits.
- Los `WeakTensor` se crearán y usarán para las [API unarias y binarias de WeakTensor](#weak_tensor_apis).


### Resultados de promoción más consistentes y predecibles

Usar un [sistema basado en retículo (lattice)](#lattice_system_design) permite que la promoción de tipos nueva produzca resultados de promoción de tipos que son consistentes y predecibles.

#### Promoción de tipos anterior

Con la promoción de tipos anterior, al cambiar el orden de las operaciones se producen resultados inconsistentes.

In [ ]:
# Setup
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="legacy")
a = np.array(1, dtype=np.int8)
b = tf.constant(1)
c = np.array(1, dtype=np.float16)

In [ ]:
# (a + b) + c throws an InvalidArgumentError.
try:
  tf.add(tf.add(a, b), c)
except tf.errors.InvalidArgumentError as e:
  print(f'{type(e)}: {e}')  # InvalidArgumentError

In [ ]:
# (b + a) + c returns an i32 result.
tf.add(tf.add(b, a), c)  # <tf.Tensor: shape=(), dtype=int32, numpy=3>

#### Promoción de tipos nueva

La promoción de tipos nueva produce resultados consistentes independientemente del orden.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = np.array(1, dtype=np.int8)
b = tf.constant(1)
c = np.array(1, dtype=np.float16)

In [ ]:
# (a + b) + c returns a f16 result.
tf.add(tf.add(a, b), c)  # <tf.Tensor: shape=(), dtype=float16, numpy=3.0>

In [ ]:
# (b + a) + c also returns a f16 result.
tf.add(tf.add(b, a), c)  # <tf.Tensor: shape=(), dtype=float16, numpy=3.0>

### Riesgo reducido de ampliación de bits

#### Promoción de tipos anterior

La promoción de tipos anterior, con frecuencia, producía resultados en 64 bits.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="legacy")

In [ ]:
np.array(3.2, np.float16) + tf.constant(1, tf.int8) + tf.constant(50)  # <tf.Tensor: shape=(), dtype=float64, numpy=54.19921875>

#### Promoción de tipos nueva

La promoción de tipos nueva devuelve resultados con el número mínimo de bits necesario.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")

In [ ]:
np.array(3.2, np.float16) + tf.constant(1, tf.int8) + tf.constant(50)  # <tf.Tensor: shape=(), dtype=float16, numpy=54.2>

### Los métodos mágicos (dunder) matemáticos en tf.Tensor

Todos los métodos mágicos (dunder) matemáticos en `tf.Tensor` seguirán la misma promoción de tipos nueva.

In [ ]:
-tf.constant(5)  # <tf.Tensor: shape=(), dtype=int32, numpy=-5, weak=True>

In [ ]:
tf.constant(5, tf.int16) - tf.constant(1, tf.float32)  # <tf.Tensor: shape=(), dtype=float32, numpy=4.0>

### Operaciones en el lugar con tf.Variable

Las conversiones implícitas se podrán hacer en operaciones en el lugar con `tf.Variable`.

**Nota**: no se permitirá ninguna promoción que dé como resultado un tipo d que sea diferente del tipo d original de la variable. El motivo es que `tf.Variable` no puede cambiar su tipo d.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.Variable(10, tf.int32)
a.assign_add(tf.constant(5, tf.int16))  # <tf.Variable shape=() dtype=int32, numpy=15>

### Conversiones implícitas con tf.constant

En la promoción de tipos anterior, para `tf.constant` se requería un tensor de entrada que tuviera el mismo tipo d del argumento. Sin embargo, en la promoción de tipos nueva, convertimos implícitamente al tensor al tipo d especificado.

In [ ]:
tnp.experimental_enable_numpy_behavior(dtype_conversion_mode="all")
a = tf.constant(10, tf.int16)
tf.constant(a, tf.float32)  # <tf.Tensor: shape=(), dtype=float32, numpy=10.0>

### Arreglo TF-NumPy

`tnp.array` propone por defecto `i32*` y `f32*` para las entradas de Python con la promoción de tipos nueva.

In [ ]:
tnp.array(1)  # <tf.Tensor: shape=(), dtype=int32, numpy=1, weak=True>

In [ ]:
tnp.array(1.0)  # <tf.Tensor: shape=(), dtype=int32, numpy=1, weak=True>

##Inferencia de tipo de entrada

Así es como los diferentes tipos de entradas se infieren con la promoción de tipos nueva.

- `tf.Tensor`: como `tf.Tensor` tiene una propiedad de tipo d, no hacemos más inferencias.
- Tipos NumPy: incluye los tipos como `np.array(1)`, `np.int16(1)` y `np.float`. Como las entradas NumPy tienen una propiedad de tipo d, tomamos la propiedad de tipo d como el tipo de inferencia de resultado. Tenga en cuenta que NumPy propone por defecto `i64` y `f64`.
- Los tipos escalares o anidados de Python: incluye tipos como `1`, `[1, 2, 3]` y `(1.0, 2.0)`.
    - `int` de Python se infiere como `i32*`.
    - `float` de Python se infiere como `f32*`.
    - `complex` de Python se infiere como `c128*`.
- Si la entrada no cae en ninguna de las categorías anteriores sino que tiene una propiedad de tipo d, tomamos la propiedad de tipo d como el tipo de inferencia resultante.

# Lecturas adicionales

La promoción de tipos nueva se parece mucho a la promoción de tipos de JAX-NumPy. Si desea conocer más detalles acerca de la promoción de tipos nueva y de las opciones de diseño, consulte los recursos que se encuentran a continuación.

- [Semántica de la promoción de tipos en JAX](https://jax.readthedocs.io/en/latest/type_promotion.html)
- [Diseño de la semántica de la promoción de tipos para JAX](https://jax.readthedocs.io/en/latest/jep/9407-type-promotion.html)
- [Semántica de la promoción de tipos en TF-NumPy anterior](https://www.tensorflow.org/guide/tf_numpy#type_promotion)


# Referencias

<a name="weak_tensor_apis">
</a>

## API compatibles con WeakTensor

A continuación, se encuentra una lista de las API compatibles con `WeakTensor`.

Para una op (operación) unaria, significa que si se pasa una entrada con tipo de usuario no especificado, devolverá un `WeakTensor`.

Para una op binaria, se seguirá la tabla de promociones que se encuentra [aquí](#promotion_table). Puede devolver un `WeakTensor` o no hacerlo, dependiendo del resultado de la promoción de las dos entradas.

**Nota**: se admiten todas las operaciones matemáticas (`+`, `-`, `*`, ...).

- `tf.bitwise.invert`
- `tf.clip_by_value`
- `tf.debugging.check_numerics`
- `tf.expand_dims`
- `tf.identity`
- `tf.image.adjust_brightness`
- `tf.image.adjust_gamma`
- `tf.image.extract_patches`
- `tf.image.random_brightness`
- `tf.image.stateless_random_brightness`
- `tf.linalg.diag`
- `tf.linalg.diag_part`
- `tf.linalg.matmul`
- `tf.linalg.matrix_transpose`
- `tf.linalg.tensor_diag_part`
- `tf.linalg.trace`
- `tf.math.abs`
- `tf.math.acos`
- `tf.math.acosh`
- `tf.math.add`
- `tf.math.angle`
- `tf.math.asin`
- `tf.math.asinh`
- `tf.math.atan`
- `tf.math.atanh`
- `tf.math.ceil`
- `tf.math.conj`
- `tf.math.cos`
- `tf.math.cosh`
- `tf.math.digamma`
- `tf.math.divide_no_nan`
- `tf.math.divide`
- `tf.math.erf`
- `tf.math.erfc`
- `tf.math.erfcinv`
- `tf.math.erfinv`
- `tf.math.exp`
- `tf.math.expm1`
- `tf.math.floor`
- `tf.math.floordiv`
- `tf.math.floormod`
- `tf.math.imag`
- `tf.math.lgamma`
- `tf.math.log1p`
- `tf.math.log_sigmoid`
- `tf.math.log`
- `tf.math.multiply_no_nan`
- `tf.math.multiply`
- `tf.math.ndtri`
- `tf.math.negative`
- `tf.math.pow`
- `tf.math.real`
- `tf.math.real`
- `tf.math.reciprocal_no_nan`
- `tf.math.reciprocal`
- `tf.math.reduce_euclidean_norm`
- `tf.math.reduce_logsumexp`
- `tf.math.reduce_max`
- `tf.math.reduce_mean`
- `tf.math.reduce_min`
- `tf.math.reduce_prod`
- `tf.math.reduce_std`
- `tf.math.reduce_sum`
- `tf.math.reduce_variance`
- `tf.math.rint`
- `tf.math.round`
- `tf.math.rsqrt`
- `tf.math.scalar_mul`
- `tf.math.sigmoid`
- `tf.math.sign`
- `tf.math.sin`
- `tf.math.sinh`
- `tf.math.softplus`
- `tf.math.special.bessel_i0`
- `tf.math.special.bessel_i0e`
- `tf.math.special.bessel_i1`
- `tf.math.special.bessel_i1e`
- `tf.math.special.bessel_j0`
- `tf.math.special.bessel_j1`
- `tf.math.special.bessel_k0`
- `tf.math.special.bessel_k0e`
- `tf.math.special.bessel_k1`
- `tf.math.special.bessel_k1e`
- `tf.math.special.bessel_y0`
- `tf.math.special.bessel_y1`
- `tf.math.special.dawsn`
- `tf.math.special.expint`
- `tf.math.special.fresnel_cos`
- `tf.math.special.fresnel_sin`
- `tf.math.special.spence`
- `tf.math.sqrt`
- `tf.math.square`
- `tf.math.subtract`
- `tf.math.tan`
- `tf.math.tanh`
- `tf.nn.depth_to_space`
- `tf.nn.elu`
- `tf.nn.gelu`
- `tf.nn.leaky_relu`
- `tf.nn.log_softmax`
- `tf.nn.relu6`
- `tf.nn.relu`
- `tf.nn.selu`
- `tf.nn.softsign`
- `tf.nn.space_to_depth`
- `tf.nn.swish`
- `tf.ones_like`
- `tf.realdiv`
- `tf.reshape`
- `tf.squeeze`
- `tf.stop_gradient`
- `tf.transpose`
- `tf.truncatediv`
- `tf.truncatemod`
- `tf.zeros_like`
- `tf.experimental.numpy.abs`
- `tf.experimental.numpy.absolute`
- `tf.experimental.numpy.amax`
- `tf.experimental.numpy.amin`
- `tf.experimental.numpy.angle`
- `tf.experimental.numpy.arange`
- `tf.experimental.numpy.arccos`
- `tf.experimental.numpy.arccosh`
- `tf.experimental.numpy.arcsin`
- `tf.experimental.numpy.arcsinh`
- `tf.experimental.numpy.arctan`
- `tf.experimental.numpy.arctanh`
- `tf.experimental.numpy.around`
- `tf.experimental.numpy.array`
- `tf.experimental.numpy.asanyarray`
- `tf.experimental.numpy.asarray`
- `tf.experimental.numpy.ascontiguousarray`
- `tf.experimental.numpy.average`
- `tf.experimental.numpy.bitwise_not`
- `tf.experimental.numpy.cbrt`
- `tf.experimental.numpy.ceil`
- `tf.experimental.numpy.conj`
- `tf.experimental.numpy.conjugate`
- `tf.experimental.numpy.copy`
- `tf.experimental.numpy.cos`
- `tf.experimental.numpy.cosh`
- `tf.experimental.numpy.cumprod`
- `tf.experimental.numpy.cumsum`
- `tf.experimental.numpy.deg2rad`
- `tf.experimental.numpy.diag`
- `tf.experimental.numpy.diagflat`
- `tf.experimental.numpy.diagonal`
- `tf.experimental.numpy.diff`
- `tf.experimental.numpy.empty_like`
- `tf.experimental.numpy.exp2`
- `tf.experimental.numpy.exp`
- `tf.experimental.numpy.expand_dims`
- `tf.experimental.numpy.expm1`
- `tf.experimental.numpy.fabs`
- `tf.experimental.numpy.fix`
- `tf.experimental.numpy.flatten`
- `tf.experimental.numpy.flip`
- `tf.experimental.numpy.fliplr`
- `tf.experimental.numpy.flipud`
- `tf.experimental.numpy.floor`
- `tf.experimental.numpy.full_like`
- `tf.experimental.numpy.imag`
- `tf.experimental.numpy.log10`
- `tf.experimental.numpy.log1p`
- `tf.experimental.numpy.log2`
- `tf.experimental.numpy.log`
- `tf.experimental.numpy.max`
- `tf.experimental.numpy.mean`
- `tf.experimental.numpy.min`
- `tf.experimental.numpy.moveaxis`
- `tf.experimental.numpy.nanmean`
- `tf.experimental.numpy.negative`
- `tf.experimental.numpy.ones_like`
- `tf.experimental.numpy.positive`
- `tf.experimental.numpy.prod`
- `tf.experimental.numpy.rad2deg`
- `tf.experimental.numpy.ravel`
- `tf.experimental.numpy.real`
- `tf.experimental.numpy.reciprocal`
- `tf.experimental.numpy.repeat`
- `tf.experimental.numpy.reshape`
- `tf.experimental.numpy.rot90`
- `tf.experimental.numpy.round`
- `tf.experimental.numpy.signbit`
- `tf.experimental.numpy.sin`
- `tf.experimental.numpy.sinc`
- `tf.experimental.numpy.sinh`
- `tf.experimental.numpy.sort`
- `tf.experimental.numpy.sqrt`
- `tf.experimental.numpy.square`
- `tf.experimental.numpy.squeeze`
- `tf.experimental.numpy.std`
- `tf.experimental.numpy.sum`
- `tf.experimental.numpy.swapaxes`
- `tf.experimental.numpy.tan`
- `tf.experimental.numpy.tanh`
- `tf.experimental.numpy.trace`
- `tf.experimental.numpy.transpose`
- `tf.experimental.numpy.triu`
- `tf.experimental.numpy.vander`
- `tf.experimental.numpy.var`
- `tf.experimental.numpy.zeros_like`